In [2]:
import stageemi
import stageemi.dev.visu as dev
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import stageemi.dev.decorator_map as dm
import ipywidgets as widg 
import ipyleaflet as ipyl
import datetime as dt
import os 
from ipywidgets import Text, HTML
from ipyleaflet import WidgetControl
import pandas as pd
from datetime import datetime
import glob 

In [33]:
def conversion(ds,name):
    """convert wwmf into wme (compas) or w1 (agat) code"""
    
    file_CodesWWMF= '../utils/CodesWWMF.csv'
    df_WWMF = pd.read_csv(file_CodesWWMF,usecols = (0,1,2,3,6,7),sep=',')
        
    if name=="compas":
        var_name="wme_arr"
        col_name="Code WME"
        ds[var_name]=ds.unknown
        
    elif name=="agat":
        var_name="w1_arr"
        col_name="Code W1"
        ds[var_name]=ds.unknown       
        
    elif name=="compas_asym":
        var_name="wme_asym_arr"
        col_name="Code WME"
        ds[var_name]=ds.unknown
        
    elif name=="agat_asym":
        var_name="w1_asym_arr"
        col_name="Code W1"
        ds[var_name]=ds.unknown 

    for iwwmf,wwmf in enumerate(df_WWMF["Code WWMF"]):
        #print(wwmf,df_WWMF["Code WME"][iwwmf])
        ds[var_name]=ds[var_name].where(ds.unknown!=wwmf,df_WWMF[col_name][iwwmf])
        
    return ds

def distance(ds,name,**options):
    """calculate the distance between all possible temps sensibles (wme) and the temps sensibles of the zone"""
    
    if name == "compas":
        fname_dist = '../utils/distance_compas.csv'
        df_dist = pd.read_csv(fname_dist,sep=',')
        var_name="wme_arr"
        varsh="wme_c_"
        
    elif name == "agat":
        fname_dist = '../utils/distance_agat.csv'
        df_dist = pd.read_csv(fname_dist,sep=',')  
        var_name="w1_arr"
        varsh="w1_c_"
        
    elif name == "compas_asym":
        fname_dist = '../utils/distance_compas_asym.csv'
        df_dist = pd.read_csv(fname_dist,sep=',')
        var_name="wme_asym_arr"
        varsh="wme_asym_c_"
        
    elif name == "agat_asym":
        fname_dist = '../utils/distance_agat_asym.csv'
        df_dist = pd.read_csv(fname_dist,sep=',')  
        var_name="w1_asym_arr"
        varsh="w1_asym_c_"
    
        
    if options.get("action") == "test":
        """used to test over few pixels only in debug mode"""
        for iwme,wme in enumerate(df_dist):
            if iwme>0 and iwme<5:
                # initialize ds["1"] etc
                ds[wme]=ds.wme_arr
                for iiwme,wwme in enumerate(df_dist):
                    if iiwme>0:
                        #print(wme,wwme,iiwme,iwme)
                        #print(df_dist.iloc[iiwme-1,iwme])
                        # for a given wme (e.g. "1") every value in ds["1"] is replaced by the ditance btw wme and wwme
                        ds[wme]=ds[wme].where(ds.wme_arr!=int(wwme),df_dist.iloc[iiwme-1,iwme])  
                        
    else:
        for iw,w in enumerate(df_dist): 
            if iw>0:
                ds[varsh+w]=ds.wme_arr
                for iiw,ww in enumerate(df_dist):
                    if iiw>0:
                        ds[varsh+w]=ds[varsh+w].where(ds[var_name]!=int(ww),df_dist.iloc[iiw-1,iw])       
                    
    return ds  

def shortest_distance_temps_sensible(ds,name):
       
    if name=="compas": 
        varsh="wme_c_"
    elif name=="agat": 
        varsh="w1_c_"
    elif name=="compas_asym": 
        varsh="wme_asym_c_"
    elif name=="agat_asym": 
        varsh="w1_asym_c_"
    
    """find all newly added variables linked to wme or w1 resulting from the distance calulation"""
    allvar=list(ds.data_vars)
    list_w=[allvar[i] for i in np.where([varsh in s for s in allvar])[0]]
    ncodes=len(list_w)
    
    nstep=1#ds.dims["step"]    
    best_w=np.asarray(np.ones((1,nstep))*np.nan)  
    
    #add-on
    second_best_w=np.asarray(np.ones((1,nstep))*np.nan)
    best_w_dist=np.asarray(np.ones((1,nstep))*np.nan)  
    second_best_w_dist=np.asarray(np.ones((1,nstep))*np.nan)
    
    # start comment here
    #for istep in range(nstep):
    istep=0
        
    """used for debug"""
    dist_w=np.asarray(np.ones((1,ncodes))*np.nan) 

    best_w[0]=list_w[np.asarray([np.sum(ds[w]) for iw,w in enumerate(list_w)]).argmin()][len(varsh)::]
    dist_w[0]=np.asarray([np.sum(ds[w]) for iw,w in enumerate(list_w)])

    ds.attrs[name+" (step"+str(istep)+")"]=dist_w[0]

    #add-on (20200313 teletravail day - end of EMI)
    #xxxx_dist_best_class + xxxx_second_best_class + xxxx_dist_second_best_class + (xxxx_mean_dist + xxxx_std_dist)
    second_best_w[0]=list_w[(np.argsort(np.asarray([np.sum(ds[w]) for iw,w in enumerate(list_w)])))[1]][len(varsh)::]
    best_w_dist[0]=np.sort(dist_w[0])[0]
    second_best_w_dist[0]=np.sort(dist_w[0])[1]
        
    # end comment here
    
    """write the best wme code for each time step """
    ds.attrs[name]=best_w[0]    
    
    #add-on
    ds.attrs[name+" second-best-class"]=second_best_w[0]
    ds.attrs[name+" best-distance"]=best_w_dist[0]
    ds.attrs[name+" second-best-distance"]=second_best_w_dist[0]
    
    return ds    
    
def calculate_distance(ds,name):
   
    ds=conversion(ds,name)
    ds=distance(ds,name)
    ds=shortest_distance_temps_sensible(ds,name)
    
    return ds

In [ ]:
def store_results(df_scores,df,ds_masked,**options):
    """store coherency results in df_scores"""
    if options.get("action") == "plot": 
        fig,axes = plt.subplots(nrows=8,ncols = 2,figsize=(30,60))
        ax = axes.flat
    
    for i in range(ds_masked.dims['step']):
        istep=i
        val = ds_masked.unknown.isel(step=istep).copy()
        code,leg = majoritaire(val.values, code_WWMF,legende_WWMF)
        
        code_WME=int(ds_masked.attrs["compas"][istep])        
        if code_WME==17 or code_WME>19:
            print("Warning, one code WME is either equal to 17 or greater than the max 19: code WME=",code_WME,"in step:",istep)            
        else:
            leg_WME=np.unique(df[df['Code WME']==code_WME]['Legende WME'])[0]

        code_W1=int(ds_masked.attrs["agat"][istep])
        if code_W1==4 or code_W1==12 or code_W1==15 or code_W1==25 or code_W1==27 or code_W1>28:
            print("Warning, one code W1 is greater than the max 26: code W1=",code_W1,"in step:",istep)            
        else:
            leg_W1=np.unique(df[df['Code W1']==code_W1]['Legende W1'])[0]
                    
        #add-on
        code_WME_second_best_class=int(ds_masked.attrs["compas second-best-class"][istep])
        code_WME_best_distance=int(ds_masked.attrs["compas best-distance"][istep])
        code_WME_second_best_distance=int(ds_masked.attrs["compas second-best-distance"][istep])
        
        code_W1_second_best_class=int(ds_masked.attrs["agat second-best-class"][istep])
        code_W1_best_distance=int(ds_masked.attrs["agat best-distance"][istep])
        code_W1_second_best_distance=int(ds_masked.attrs["agat second-best-distance"][istep])
        
        code_WME_asym=int(ds_masked.attrs["compas_asym"][istep]) 
        code_WME_asym_second_best_class=int(ds_masked.attrs["compas_asym second-best-class"][istep])
        code_WME_asym_best_distance=int(ds_masked.attrs["compas_asym best-distance"][istep])
        code_WME_asym_second_best_distance=int(ds_masked.attrs["compas_asym second-best-distance"][istep])
        
        code_W1_asym=int(ds_masked.attrs["agat_asym"][istep])
        code_W1_asym_second_best_class=int(ds_masked.attrs["agat_asym second-best-class"][istep])
        code_W1_asym_best_distance=int(ds_masked.attrs["agat_asym best-distance"][istep])
        code_W1_asym_second_best_distance=int(ds_masked.attrs["agat_asym second-best-distance"][istep])
        
        
        time_step=str(np.asarray(ds_masked.time[istep].dt.strftime('%Y %m %d %H %M')))
        codes_coherency(df,df_scores,time_step,code,code_WME,code_W1,code_WME_asym,code_W1_asym,
                                code_WME_second_best_class,code_W1_second_best_class,code_WME_asym_second_best_class,code_W1_asym_second_best_class,
                                code_WME_best_distance,code_W1_best_distance,code_WME_asym_best_distance,code_W1_asym_best_distance,    
                                code_WME_second_best_distance,code_W1_second_best_distance,code_WME_asym_second_best_distance,code_W1_asym_second_best_distance)
        
        if options.get("action") == "plot":        
            val.plot.imshow(ax=ax[i],levels=range(0,90))    
            ax[i].set_title("WWMF:"+leg+' '+str(code)+" - WME:"+leg_WME+' '+str(code_WME)+"\nW1:"+leg_W1+' '+str(code_W1)+" Coherency= "+str(df_scores.loc[time_step]["WWMF vs (WME and W1)"]),fontsize=30)
            ax[i].set_xlabel('longitude',fontsize=20)
            ax[i].set_ylabel('latitude',fontsize=20)
    
        
    if options.get("action") == "plot": 
        plt.tight_layout()
        
    return df_scores 

In [8]:
path="../WWMF/"
fname=sorted(glob.glob(path+"*"+'20200126000000'+"*"))[0]
echeance=34

In [6]:
zone_dpt="29"
fname_mask='../GeoData/zones_sympo_multiples/'+zone_dpt+'_mask_zones_sympos.nc'
da_mask=xr.open_dataarray(fname_mask)

da_mask['latitude']=da_mask['latitude'].round(5)
da_mask['latitude']=da_mask['latitude'].round(5)

zones=da_mask.id.values.tolist()
#zones

In [34]:
for izone, zone in enumerate(zones[0:2]):
    
    if zone != "departement":
        
        ds=xr.open_dataset(fname)
        
        """solve the bug observed when multiplying the mask by the dataset"""
        ds['latitude']=ds['latitude'].round(5)
        ds['longitude']=ds['longitude'].round(5)
        
        ds_masked=ds.isel(step=echeance)*ds_mask.isel(id=izone)
        
        ds_masked=calculate_distance(ds_masked,"compas")
        ds_masked=calculate_distance(ds_masked,"agat")
        ds_masked=calculate_distance(ds_masked,"compas_asym")
        ds_masked=calculate_distance(ds_masked,"agat_asym")
        
        
        
        print(ds_masked)
        #ds_sub.unknown.plot.imshow()
    
    

<xarray.Dataset>
Dimensions:            (latitude: 116, longitude: 153)
Coordinates:
  * latitude           (latitude) float64 48.83 48.82 48.81 ... 47.7 47.69 47.68
  * longitude          (longitude) float64 -4.91 -4.9 -4.89 ... -3.41 -3.4 -3.39
    surface            int64 ...
    heightAboveGround  int64 ...
    id                 <U4 '2901'
Data variables:
    unknown            (latitude, longitude) float32 nan nan nan ... nan nan nan
    wme_arr            (latitude, longitude) float32 nan nan nan ... nan nan nan
    wme_c_1            (latitude, longitude) float32 nan nan nan ... nan nan nan
    wme_c_2            (latitude, longitude) float32 nan nan nan ... nan nan nan
    wme_c_3            (latitude, longitude) float32 nan nan nan ... nan nan nan
    wme_c_4            (latitude, longitude) float32 nan nan nan ... nan nan nan
    wme_c_5            (latitude, longitude) float32 nan nan nan ... nan nan nan
    wme_c_6            (latitude, longitude) float32 nan nan nan ... n